In [ ]:
import tensorflow as tf
from tensorflow import keras
import os, argparse
import cv2, spacy, numpy as np
from sklearn.externals import joblib
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import collections
from collections import Counter
import sklearn
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Dropout, LSTM, Flatten, Embedding, merge,TimeDistributed
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

import h5py
from sklearn.metrics import confusion_matrix
np.random.seed(345)
from sklearn.metrics import accuracy_score 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving df.txt to df.txt


In [ ]:
data=open("df.txt")
split_lines=data.read().split("\n")

In [ ]:

movie_id=[]
genre=[]
description=[]

for i in range(41793):
    genre.append(split_lines[i].split("|")[1].lower())
    description.append(split_lines[i].split("|")[2].lower())
    movie_id.append(split_lines[i].split("|")[0].lower())


In [ ]:
#movie_id

In [ ]:
import collections

# genre
#x

In [ ]:

filename="df.txt" 
#filename1="D:/Imageclef/glove.6B.100d.txt" # for glove embedding vector

# dictionary define to separate movie id, description and genre

def separate(filename):
     movie_id=[]
     genre=[]
     description=[]
     dictionary_plane=dict()

    

     for i in range(41793):
           genre.append(split_lines[i].split("|")[1].lower())
           description.append(split_lines[i].split("|")[2].lower())
           movie_id.append(split_lines[i].split("|")[0].lower())


        
     dictionary_plane["movie_id"]=movie_id
     dictionary_plane["genre"]=genre
     dictionary_plane["description"]=description
    
    
    
        

        
     return dictionary_plane  


In [ ]:
des=separate(filename).get("description")



In [ ]:
def my_token(description):
    t=tf.keras.preprocessing.text.Tokenizer()
    t.fit_on_texts(description)
    word_index=t.word_index
    word_index
    return word_index 


In [ ]:
token_data=my_token(des)
len(token_data)

152691

In [ ]:
def my_sequence(description):     
    t=tf.keras.preprocessing.text.Tokenizer()
    t.fit_on_texts(description)
    seq_ques=t.texts_to_sequences(description)
    return seq_ques  # return list of question. each question now represented by integer token



In [ ]:
 seq_data=my_sequence(des)


In [ ]:

def my_padding(seq_data):
    pad_ques=pad_sequences(seq_data)
    return pad_ques
   



In [ ]:
pad_data=my_padding(seq_data)
pad_data.shape

(41793, 2605)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# multilabel_binarizer = MultiLabelBinarizer()
# multilabel_binarizer.fit(movies_new['genre_new'])

# # transform target variable
# y = multilabel_binarizer.transform(movies_new['genre_new'])


def one_hot_code(genre):
    y_array=np.array(genre)
    multilabel_binarizer=MultiLabelBinarizer()
    i_encoded=multilabel_binarizer.fit(y_array)
    i_transformed=i_encoded.transform(y_array)

    # onehot_encoder = MultiLabelBinarizer(sparse_output=False)
    # integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    # onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return i_transformed



In [ ]:
# ohe=one_hot_code(separate(filename).get("genre"))
ohe=one_hot_code(separate(filename).get("genre"))


In [ ]:
ohe.shape


(41793, 38)

In [ ]:
# def embedding(filename1,token_data): 
#     word_index=token_data # call my_token function
#     embeddings_index = dict()
#     f = open(filename1,encoding='utf8') # call glove vector text file
#     for line in f:
#         values=line.split() # split the each line in text file
#         word = values[0] # first index associate with word and othe other indexs represent embedding vector associated with that word 
#         coefs = np.asarray(values[1:], dtype='float32') 
#         embeddings_index[word] = coefs
#     f.close()

#     vocab_size=len(my_token(filename))
#     embedding_matrix=np.zeros((vocab_size+1,100)) # define embedding matrix

#     for word,i in word_index.items():                # create our embedding matrix for each word of questions.
#         embedding_vector = embeddings_index.get(word)
#         if embedding_vector is not None:
#             embedding_matrix[i] = embedding_vector
#             return embedding_matrix



In [ ]:
# emb=embedding(filename1,token_data)

In [ ]:
model = Sequential()
model.add(Embedding(152691, 200, input_length=2605))
model.add(LSTM(units=100, return_sequences=True, input_shape=(2605, 200)))
model.add(Dropout(.3))  
model.add(Dropout(.3))
model.add(Dense(38, activation="softmax"))  
 

model.summary() 





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2605, 200)         30538200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 2605, 100)         120400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2605, 100)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 38)                3838      
T

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])


In [ ]:
fit=model.fit(pad_data,ohe,batch_size=1000,epochs=5)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





19712/41793 [=============>................] - ETA: 27:54 - loss: 51.8870 - acc: 0.0519

In [ ]:
model_c = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model_c.add(Embedding(152691,
                    32,
                    input_length=2605))
model_c.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model_c.add(Conv1D(filters=64,
                 kernel_size=3,
                
                 activation='relu'))
# we use max pooling:
model_c.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model_c.add(Dense(10))
model_c.add(Dropout(0.2))
model_c.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model_c.add(Dense(38))
model_c.add(Activation('softmax'))

model_c.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2605, 32)          4886112   
_________________________________________________________________
dropout_5 (Dropout)          (None, 2605, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2603, 64)          6208      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
_________________________________________________________________
dropout_6 (Dropout)          (None, 10)                0         
_________________________________________________________________
activation_3 (Activation)    (None, 10)               

In [ ]:
model_c.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model_c.fit(pad_data, ohe,
          batch_size=2056,
          epochs=50)

Epoch 1/50
41793/41793 [==============================] - 18s 429us/step - loss: 49.7570 - acc: 0.1718
Epoch 2/50
41793/41793 [==============================] - 17s 403us/step - loss: 49.6686 - acc: 0.1805
Epoch 3/50
41793/41793 [==============================] - 17s 405us/step - loss: 49.5717 - acc: 0.1965
Epoch 4/50
41793/41793 [==============================] - 17s 407us/step - loss: 49.4749 - acc: 0.2219
Epoch 5/50
41793/41793 [==============================] - 17s 406us/step - loss: 49.3819 - acc: 0.2294
Epoch 6/50
41793/41793 [==============================] - 17s 407us/step - loss: 49.2976 - acc: 0.2424
Epoch 7/50
41793/41793 [==============================] - 17s 406us/step - loss: 49.2134 - acc: 0.2447
Epoch 8/50
41793/41793 [==============================] - 17s 406us/step - loss: 49.1305 - acc: 0.2415
Epoch 9/50
41793/41793 [==============================] - 17s 406us/step - loss: 49.0688 - acc: 0.2420
Epoch 10/50
41793/41793 [==============================] - 17s 405us/step

In [ ]:
# score_c = model_c.evaluate(xval_tfidf, yval, batch_size=512)